In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
DATE_FORMAT = '%m.%Y'

In [3]:
def safe_mkdf(path, cols):
    dateparse = lambda x: pd.datetime.strptime(x, '%m.%Y')
    if os.path.exists(path):
        df = pd.read_csv(path, index_col=0, parse_dates = [1], date_parser=dateparse)
    else:
        df = pd.DataFrame(columns=cols)
    return df

In [4]:
class Hist_df():
    hist_columns = ['Date', 'Amount']
    
    
    def __init__(self, path):
        self.path = path
        self.df = safe_mkdf(path, Hist_df.hist_columns)
        
    def add_row(self, string_date, amnt):
        date_tm = pd.to_datetime(string_date, format=DATE_FORMAT)
        index = self.df.index.size
        self.df.loc[index] = pd.Series({Hist_df.hist_columns[0]: date_tm, Hist_df.hist_columns[1]: amnt})
        self.df.to_csv(self.path, date_format= DATE_FORMAT)
        
    def sort(self):
        self.df = self.df.sort_values(by = 'Date')
        self.df.index = np.arange(self.df.index.size)
        
    def get_grouped(self):
        return self.df.groupby(['Date']).sum()
        

In [5]:
def check_dates(hist_df):
    return not hist_df.df['Date'].duplicated().any()

In [115]:
class Investment():
    
    def safe_mkdir(self, path):
        if not os.path.exists(path):
            os.makedirs(path)
            
    def __init__(self, path):
        self.safe_mkdir(os.path.expanduser(path))
        
        self._stat_hist_filename = os.path.join(os.path.expanduser(path),'stat_hist.csv')
        
        self._deps_hist_filename = os.path.join(os.path.expanduser(path),'deps_hist.csv')
        
        self._withs_hist_filename = os.path.join(os.path.expanduser(path),'withs_hist.csv')
        
        self.stat_hist_df = Hist_df(self._stat_hist_filename)
        self.deps_hist_df = Hist_df(self._deps_hist_filename)
        self.withs_hist_df = Hist_df(self._withs_hist_filename)
        self.grouped_stat_df = pd.DataFrame()
        self.grouped_stat_diff_df = pd.DataFrame()
        self.grouped_withs_hist_df = pd.DataFrame()
        self.grouped_deps_hist_df = pd.DataFrame()
        
    def add_deposit(self, string_date, amnt):
        self.deps_hist_df.add_row(string_date, amnt)
        self.deps_hist_df.sort()
        self.grouped_deps_hist_df = self.deps_hist_df.get_grouped()
        
    def add_state(self, string_date, amnt):
        self.stat_hist_df.add_row(string_date, amnt)
        self.stat_hist_df.sort()
        self.grouped_stat_diff_df = self.stat_hist_df.df.set_index('Date').diff()
        if not check_dates(self.stat_hist_df):
            print("Warining! There is duplicates in state history")
            
    def add_withdrawal(self, string_date, amnt):
        self.withs_hist_df.add_row(string_date, amnt)
        self.withs_hist_df.sort()
        self.grouped_withs_hist_df = self.withs_hist_df.get_grouped()
        
    def build(self):
        self.stat_hist_df.sort()
        self.grouped_stat_df = self.stat_hist_df.df.set_index('Date').iloc[:-1]
        self.grouped_stat_diff_df = self.stat_hist_df.df.set_index('Date').diff().shift(-1).iloc[:-1]
        self.deps_hist_df.sort()
        self.grouped_deps_hist_df = self.deps_hist_df.get_grouped()
        self.withs_hist_df.sort()
        self.grouped_withs_hist_df = self.withs_hist_df.get_grouped()
        indexes = self.grouped_stat_diff_df.index.union(
            self.grouped_withs_hist_df.index).union(self.grouped_deps_hist_df.index)
        if len(indexes) > 0:
            first_i = indexes[0]
            last_i = indexes[-1]
            self.indexed_stat_diff_df = self.grouped_stat_diff_df.reindex(indexes, fill_value=0)
            self.indexed_withs_df = self.grouped_withs_hist_df.reindex(indexes, fill_value=0)
            self.indexed_deps_df = self.grouped_deps_hist_df.reindex(indexes, fill_value=0)
        
    def get_profits(self):
        return self.indexed_stat_diff_df + \
                self.indexed_withs_df - \
                self.indexed_deps_df
            
    def get_returns(self):
        return self.get_profits() / self.grouped_stat_df
    
    def get_returns_anual(self):
        return self.get_returns() * 12
    
    def get_rolling_return(self, window):
        return self.get_profits().rolling(window).sum() / self.grouped_stat_df.rolling(window).mean()
    
    def get_rolling_return_anual(self, window):
        return self.get_rolling_return(window) / window * 12

In [192]:
invest = Investment('./invest')

# invest.add_deposit('03.2017', 5)
# invest.add_withdrawal('04.2017', 70)

In [193]:
invest.add_state('01.2017', 142840.62)
invest.add_state('02.2017', 143995.56)

In [194]:
invest.add_state('03.2017', 193245.68)

In [195]:
invest.add_deposit('02.2017', 50000)

In [196]:
invest.add_state('04.2017', 195490.97)

In [197]:
invest.add_state('05.2017', 196769.01)

In [198]:
invest.add_state('06.2017', 197372.56)

In [199]:
invest.add_state('07.2017', 198200.73)

In [200]:
invest.add_state('08.2017', 250631.00)
invest.add_deposit('07.2017', 50000)

In [201]:
invest.add_state('09.2017', 252846.68)
invest.add_withdrawal('08.2017', 11700)

In [203]:
invest.add_state('10.2017', 251136.59)

In [205]:
invest.build()
print("Profits", invest.get_profits())
print("Returns", invest.get_returns())
print("Returns_anual", invest.get_returns_anual())
print("RollRet", invest.get_rolling_return(9))
print("RollRet_anual", invest.get_rolling_return_anual(9))

Profits               Amount
Date                
2017-01-01   1154.94
2017-02-01   -749.88
2017-03-01   2245.29
2017-04-01   1278.04
2017-05-01    603.55
2017-06-01    828.17
2017-07-01   2430.27
2017-08-01  13915.68
2017-09-01  -1710.09
Returns               Amount
Date                
2017-01-01  0.008086
2017-02-01 -0.005208
2017-03-01  0.011619
2017-04-01  0.006538
2017-05-01  0.003067
2017-06-01  0.004196
2017-07-01  0.012262
2017-08-01  0.055523
2017-09-01 -0.006763
Returns_anual               Amount
Date                
2017-01-01  0.097026
2017-02-01 -0.062492
2017-03-01  0.139426
2017-04-01  0.078451
2017-05-01  0.036808
2017-06-01  0.050352
2017-07-01  0.147140
2017-08-01  0.666271
2017-09-01 -0.081160
RollRet               Amount
Date                
2017-01-01       NaN
2017-02-01       NaN
2017-03-01       NaN
2017-04-01       NaN
2017-05-01       NaN
2017-06-01       NaN
2017-07-01       NaN
2017-08-01       NaN
2017-09-01  0.101594
RollRet_anual               Amount
Dat

In [38]:
r=pd.DataFrame(columns=['x', 'y'])
r.loc[0] = [1,2]
r

,x,y
0,1,2
